In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import os
import pandas as pd
import re
import requests
import seaborn as sns
import descartes
import geopandas as gpd

from arcgis.geocoding import geocode
from arcgis.gis import GIS
from dotenv import load_dotenv
from functools import reduce
from shapely.geometry import Point, Polygon
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.preprocessing import StandardScaler, LabelEncoder, RobustScaler
from sklearn.svm import SVC, SVR
from textwrap import wrap
from xgboost import XGBClassifier, XGBRegressor, plot_importance
from yellowbrick.classifier import ROCAUC, ConfusionMatrix

plt.style.use("ggplot")
sns.set_context("talk")

In [2]:
load_dotenv("")
censuskey = os.getenv("censuskey")

In [11]:
var_description = pd.DataFrame(pd.DataFrame(
    requests.get("https://api.census.gov/data/2016/acs/acs5/profile/variables.json")
                                            .json()["variables"])
                               .T.iloc[:, 3]
                               .str.title()
                               .str.replace("!!", " ", regex=True)
                               .str.replace("Percent Estimate ", " ", regex=True))

In [14]:
var_description.to_csv("../data/var_description.csv")

In [19]:
variables_selected = var_description.loc[((~var_description.index.str.contains(r'DP')|
                                           (var_description.index.str.contains(r'DP')&
                                            var_description.index.str.contains(r'[0-9]PE')))&
                                          ~var_description.index.str.contains(r'PR_'))].sort_index()

In [22]:
varlist_2016 = []
k = 0
for var in variables_selected.index:
    resp = requests.get(
        "https://api.census.gov/data/2016/acs/acs5/profile?get={}&for=county:*&in=state:*&key={}"
        .format(var, censuskey))
    if resp.status_code ==200:
        varlist_2016.append(pd.DataFrame(resp.json()[1:], columns=resp.json()[0]))
    k += 1
    if k%int(len(variables_selected)/10)==0:
        print("---progress: {}% completed---".format(k*10/int(len(variables_selected)/10)))

In [ ]:
api.census.gov/data/2009/acs5
api.census.gov/data/2010/acs/acs5

In [ ]:
varlist_dict = {}
for year in range(2010, 2018):
    varlist_dict[year] = []
    k = 0
    for var in variables_selected.index:
        resp = requests.get(
            "https://api.census.gov/data/{}/acs/acs5/profile?get={}&for=county:*&in=state:*&key={}"
            .format(year, var, censuskey))
        if resp.status_code ==200:
            varlist_dict[year].append(pd.DataFrame(resp.json()[1:], columns=resp.json()[0]))
        k += 1
        if k%int(len(variables_selected)/10)==0:
            print("---progress: {:0.2f}% completed for {}---".format(k*10/int(len(variables_selected)/10), year))

---progress: 10.00% completed for 2010---
---progress: 20.00% completed for 2010---
---progress: 30.00% completed for 2010---
---progress: 40.00% completed for 2010---
---progress: 50.00% completed for 2010---
---progress: 60.00% completed for 2010---
---progress: 70.00% completed for 2010---
---progress: 80.00% completed for 2010---
---progress: 90.00% completed for 2010---
---progress: 100.00% completed for 2010---
---progress: 10.00% completed for 2011---
---progress: 20.00% completed for 2011---
---progress: 30.00% completed for 2011---
---progress: 40.00% completed for 2011---
---progress: 50.00% completed for 2011---
---progress: 60.00% completed for 2011---
---progress: 70.00% completed for 2011---
---progress: 80.00% completed for 2011---
---progress: 90.00% completed for 2011---
---progress: 100.00% completed for 2011---
---progress: 10.00% completed for 2012---
---progress: 20.00% completed for 2012---
---progress: 30.00% completed for 2012---
---progress: 40.00% completed fo

In [26]:
varlist_2009

[]